### Monday, January 15, 2024

Re-run this code ... 

### Tuesday, January 9, 2024

Gonna try to run through all of this again ...

### Monday, January 8, 2024

Gonna give this another go on the hfpt_Dec14 container and using the "teknium/OpenHermes-2.5-Mistral-7B" model"

docker container run hfpt_Dec14

This notebook was originally run on a different container, so the path to the transformers folder is now found at '/root/.cache/huggingface/hub'

1) pip install trl

Running this to the end sucks up all the RAM plus all of the swap space ... and yeah, Jack was running something while I was runnin this, so that could have been part of the problem ... meh ... I will figure out how to get around this later ... but it looks good for now.


### Thursday, December 14, 2023

I got most of this notebook to run. I interrupted the training loop because it was going to take what looks like about an hour to complete and right now I want to continue with some other things. I had to drastically reduce a few of the training parameters to get this to run on the 4090. The default values generated CUDA Out of memory errors.

### Wednesday, December 13, 2023

I guess this is the first time running this. I recall bailing on this earlier because of this mysterious google.colab userdata object, but now realize it's used just to get the users api keys. 

This notebook uses the "teknium/OpenHermes-2.5-Mistral-7B" model, which I have not downloaded. I also noticed this model was also updated 11 days ago.

I tried to use this notebook against the 'mistralai/Mistral-7B-Instruct-v0.1' model, but it fails when it tries to run the tokenizer. Clearly, there is a difference in the tokenizer of the 'mistralai/Mistral-7B-Instruct-v0.1' model and the tokenizer of the "teknium/OpenHermes-2.5-Mistral-7B" model. Sigh .. Looks like I really need to download the "teknium/OpenHermes-2.5-Mistral-7B" model!


### Monday, December 11, 2023

This is sample notebook out of [this repo](https://github.com/mlabonne/llm-course/tree/main), a Large Language Model course by Maxime Labonne.

In [1]:
import os
from getpass import getpass

# enter your api key
WANDB_API_KEY = getpass("Enter your API key: ")
os.environ["WANDB_API_KEY"] = WANDB_API_KEY

In [2]:
import wandb
wandb.login(key=WANDB_API_KEY)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: robkayinto. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
# !ls /home/rob/Data2/huggingface/transformers
!ls /root/.cache/huggingface/hub


models--NousResearch--Llama-2-7b-chat-hf
models--ehartford--samantha-mistral-7b
models--mistralai--Mistral-7B-Instruct-v0.2
models--teknium--OpenHermes-2.5-Mistral-7B
models--unsloth--mistral-7b-bnb-4bit
version.txt
version_diffusers_cache.txt


In [5]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--teknium--OpenHermes-2.5-Mistral-7B 
# !rm -rf /home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-v0.1 
# !rm -rf /home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.1 
# !rm -rf /home/rob/Data2/huggingface/transformers/models--mistralai--Mistral-7B-Instruct-v0.2

In [6]:
# Load the target model if it's not already in this container.
# docker cp /home/rob/Data3/huggingface/transformers/models--teknium--OpenHermes-2.5-Mistral-7B c8324b70601d://root/.cache/huggingface/hub
# Successfully copied 14.5GB to c8324b70601d://root/.cache/huggingface/hub

In [7]:
# !ls /home/rob/Data2/huggingface/transformers
!ls /root/.cache/huggingface/hub

models--NousResearch--Llama-2-7b-chat-hf
models--ehartford--samantha-mistral-7b
models--mistralai--Mistral-7B-Instruct-v0.2
models--teknium--OpenHermes-2.5-Mistral-7B
models--unsloth--mistral-7b-bnb-4bit
version.txt
version_diffusers_cache.txt


<a href="https://colab.research.google.com/github/mlabonne/llm-course/blob/main/Fine_tune_a_Mistral_7b_model_with_DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune a Mistral-7b model with DPO

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

In [6]:
# !pip install -q datasets trl peft bitsandbytes sentencepiece wandb

In [8]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
#from google.colab import userdata
import wandb

# Defined in the secrets tab in Google Colab
# hf_token = userdata.get('huggingface')
# wb_token = userdata.get('wandb')
# wandb.login(key=wb_token)

model_name = "teknium/OpenHermes-2.5-Mistral-7B"

# I want to override the above value only because I have not yet downloaded this model, and just 
# want to know if I can get this thing to run ...
# model_name = "mistralai/Mistral-7B-Instruct-v0.1"
# NOPE! This won't work, because of the tokenizer!



In [9]:
# This is the name of the new model we will be creating ...
new_model = "NeuralHermes-2.5-Mistral-7B"

In [10]:
# import os
# from getpass import getpass

# # enter your api key
# WANDB_API_KEY = getpass("Enter your API key: ")
# os.environ["WANDB_API_KEY"] = WANDB_API_KEY

# wandb.login(key=WANDB_API_KEY)

## Format dataset

This notebook uses the [Intel/orca_dpo_pairs](https://huggingface.co/datasets/Intel/orca_dpo_pairs) dataset.

In [10]:
def chatml_format(example):
    
    # Format system
    if len(example['system']) > 0:
        message = {"role": "system", "content": example['system']}
        system = tokenizer.apply_chat_template([message], tokenize=False)
    else:
        system = ""

    # Format instruction
    message = {"role": "user", "content": example['question']}
    prompt = tokenizer.apply_chat_template([message], tokenize=False, add_generation_prompt=True)

    # This next line of code fails because there is no column in the example called 'chatgpt'
    # Format chosen answer
    # chosen = example['chatgpt'] + "<|im_end|>\n"
    # I think this is the correct solution to the above missing column ...
    chosen = example['chosen'] + "<|im_end|>\n"

    # This too fails because there is no 'llama2-13b-chat' column
    # Format rejected answer
    # rejected = example['llama2-13b-chat'] + "<|im_end|>\n"
    rejected = example['rejected'] + "<|im_end|>\n"

    return {
        "prompt": system + prompt,
        "chosen": chosen,
        "rejected": rejected,
    }

# Load dataset
dataset = load_dataset("Intel/orca_dpo_pairs")['train']

# Save columns
original_columns = dataset.column_names

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# # Format dataset
# dataset = dataset.map(
#     chatml_format,
#     remove_columns=original_columns
# )

# # Print sample
# dataset[1]

# 33.5s to download the dataset ...

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
# Format dataset
dataset = dataset.map(
    chatml_format,
    remove_columns=original_columns
)

# Print sample
dataset[1]

{'chosen': 'Midsummer House is a moderately priced Chinese restaurant with a 3/5 customer rating, located near All Bar One.<|im_end|>\n',
 'rejected': ' Sure! Here\'s a sentence that describes all the data you provided:\n\n"Midsummer House is a moderately priced Chinese restaurant with a customer rating of 3 out of 5, located near All Bar One, offering a variety of delicious dishes."<|im_end|>\n',
 'prompt': '<|im_start|>system\nYou are an AI assistant. You will be given a task. You must generate a detailed and long answer.<|im_end|>\n<|im_start|>user\nGenerate an approximately fifteen-word sentence that describes all this data: Midsummer House eatType restaurant; Midsummer House food Chinese; Midsummer House priceRange moderate; Midsummer House customer rating 3 out of 5; Midsummer House near All Bar One<|im_end|>\n<|im_start|>assistant\n'}

## Train model with DPO

In [12]:
# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)

In [13]:
%%time

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)
model.config.use_cache = False

# 5396 MiB VRAM
# 10.8s

# 5396 MiB VRAM
# 19.1s

# CPU times: user 1min 30s, sys: 1min 25s, total: 2min 56s
# Wall time: 3h 30min 9s

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

CPU times: user 28.7 s, sys: 7.75 s, total: 36.5 s
Wall time: 9.81 s


In [14]:
# # LoRA configuration
# peft_config = LoraConfig(
#     r=16,
#     lora_alpha=16,
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM",
#     target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
# )

# # Model to fine-tune
# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype=torch.float16,
#     load_in_4bit=True
# )
# model.config.use_cache = False

# Reference model
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_4bit=True
)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    bf16=True,
    report_to="wandb",
)

# These values are too high ... we run out of CUDA memory!
# Create DPO trainer
# dpo_trainer = DPOTrainer(
#     model,
#     ref_model,
#     args=training_args,
#     train_dataset=dataset,
#     tokenizer=tokenizer,
#     peft_config=peft_config,
#     beta=0.1,
#     max_prompt_length=1024,
#     max_length=1536,
# )

# Nope! Still too high!
# dpo_trainer = DPOTrainer(
#     model,
#     ref_model,
#     args=training_args,
#     train_dataset=dataset,
#     tokenizer=tokenizer,
#     peft_config=peft_config,
#     beta=0.1,
#     max_prompt_length=512,
#     max_length=1536,
# )

# Wow! Even this runs out of VRAM!!
# dpo_trainer = DPOTrainer(
#     model,
#     ref_model,
#     args=training_args,
#     train_dataset=dataset,
#     tokenizer=tokenizer,
#     peft_config=peft_config,
#     beta=0.1,
#     max_prompt_length=512,
#     max_length=1024,
# )

dpo_trainer = DPOTrainer(
    model,
    ref_model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,
    beta=0.1,
    max_prompt_length=512,
    max_length=512,
)

# 55.9s
# 11868 MiB VRAM


# # Fine-tune model with DPO
# dpo_trainer.train()

# 11868 MiB VRAM
# 1m 7.0s

# 11366 MiB VRAM

# At the conclusion of this cell, the "NeuralHermes-2.5-Mistral-7B" folder will have been created, but should be empty.

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/trl/trainer/dpo_trainer.py:294: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/12859 [00:00<?, ? examples/s]

In [14]:
# backup "teknium/OpenHermes-2.5-Mistral-7B"
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--teknium--OpenHermes-2.5-Mistral-7B /home/rob/Data3/huggingface/transformers 
# Successfully copied 14.5GB to /home/rob/Data3/huggingface/transformers


In [15]:
%%time

# This really works the 4090! 100% Usage, hovering around 320W ... !

# Fine-tune model with DPO
dpo_trainer.train()

# CPU times: user 32min 47s, sys: 15min 41s, total: 48min 28s
# Wall time: 44min 47s
# Finishes at 17298 MiB VRAM

# CPU times: user 33min 14s, sys: 16min 15s, total: 49min 29s
# Wall time: 45min 41s
# Finishes at 17298 MiB VRAM

# CPU times: user 33min 26s, sys: 15min 12s, total: 48min 39s
# Wall time: 44min 49s
# Finishes at 17298 MiB VRAM

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
Could not estimate the number of tokens of the input, floating-point operations will not be computed
/usr/local/lib/python3.8/dist-packages/torch/

Step,Training Loss
1,0.703100
2,0.697500
3,0.697900
4,0.695300
5,0.688500
6,0.690800
7,0.686900
8,0.675300
9,0.652100
10,0.650600


CPU times: user 32min 47s, sys: 15min 41s, total: 48min 28s
Wall time: 44min 47s


TrainOutput(global_step=200, training_loss=0.33921861320734026, metrics={'train_runtime': 2687.7706, 'train_samples_per_second': 1.191, 'train_steps_per_second': 0.074, 'total_flos': 0.0, 'train_loss': 0.33921861320734026, 'epoch': 0.25})

## Upload model

In [16]:
# Save artifacts
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

('final_checkpoint/tokenizer_config.json',
 'final_checkpoint/special_tokens_map.json',
 'final_checkpoint/tokenizer.model',
 'final_checkpoint/added_tokens.json',
 'final_checkpoint/tokenizer.json')

In [17]:
# Currently at 17298 MiB VRAM ...
# Flush memory 
del dpo_trainer, model, ref_model
# gc.collect()
# torch.cuda.empty_cache()

# This works! The total VRAM is now at 
# 512 MiB VRAM

In [18]:
gc.collect()

59928

In [19]:
# it is this cell which makes the VRAM to drop to 512MiB
torch.cuda.empty_cache()

In [20]:
# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 2m 6.0s

# This loads the model to CPU RAM, NOT the GPU VRAM

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
# This runs all 8 CPU's at %100! Why??

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

# 56.9s
# 48.4s


In [22]:
# At this point, the "NeuralHermes-2.5-Mistral-7B" folder is still empty!

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# This pumps in 14.5gb of files! ...

# 45.6s
# 35.1s

# Yeah, I am not going to push these to the hugging face hub ...
# Push them to the HF Hub
# model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
# tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

('NeuralHermes-2.5-Mistral-7B/tokenizer_config.json',
 'NeuralHermes-2.5-Mistral-7B/special_tokens_map.json',
 'NeuralHermes-2.5-Mistral-7B/tokenizer.model',
 'NeuralHermes-2.5-Mistral-7B/added_tokens.json',
 'NeuralHermes-2.5-Mistral-7B/tokenizer.json')

## Inference

At this point, restart the kernel to free up resources.

In [1]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
#from google.colab import userdata
import wandb

In [2]:
new_model = "NeuralHermes-2.5-Mistral-7B"

In [3]:
# Format prompt
message = [
    {"role": "system", "content": "You are a helpful assistant chatbot."},
    {"role": "user", "content": "What is a Large Language Model?"}
]
tokenizer = AutoTokenizer.from_pretrained(new_model)
prompt = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer
)

# 2m 14s
# ALL Memmory is taken plus 12GB of swap space!! 

# Damn! This takes up all CPU RAM as well as ALL the swap space and then chokes!!
# I killed this cell cuz it's gonna choke the system ...! then restarted the kernel.

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# Damn! This sucks up 100% of all 8 CPU's!!
# Generate text
sequences = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)
print(sequences[0]['generated_text'])

# I interrupted this at 20 minutes ... this is taking way too bloody long!

/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


KeyboardInterrupt: 